In [1]:
import pandas as pd
import pickle 
import sys
sys.path.append("/Ray_finrl/findrl_ray/finenv")
sys.path.append("./FinRL")
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/nasdaq74_proc_test.pkl')
train = pd.read_csv('dataset/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

In [2]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [3]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 80, State Space: 801


In [4]:
train.head()
train.dtypes

date             object
tic              object
open            float64
high            float64
low             float64
close           float64
volume          float64
day             float64
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
dtype: object

In [5]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
#env_kwargs = {
#    "hmax": 500,
#    "initial_amount": 1000000,
#    "num_stock_shares": num_stock_shares,
#    "buy_cost_pct": buy_cost_list,
 #   "sell_cost_pct": sell_cost_list,
  #  "state_space": state_space,
   # "stock_dim": stock_dimension,
   # "tech_indicator_list": INDICATORS,
   # "action_space": stock_dimension,
   # "reward_scaling": 1e-4
#}

In [6]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

In [7]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

In [8]:
from ray.rllib.algorithms.ppo import PPOConfig
ray.shutdown()
print(f"ray is being initialized")

ray is being initialized


In [9]:
#if use_ddppo:
#    config = ppo.DDPPOConfig()
#else:
#    config = ppo.PPOConfig()

# Here set envirnonment parameters if different from default
#config = config.environment(
#    env_config={'hmax':200, 'initial_amount':1000000}
#)

# Training
#config = config.training()

# Resources
#config = config.resources(
#    num_gpus=0,
#    num_cpus_per_worker=1,
#    num_gpus_per_worker=0,
#    num_trainer_workers=1,
#    num_gpus_per_trainer_worker = 1,
#    num_cpus_per_trainer_worker = 1,
#)
# Framework
#config = config.framework(
#    framework="torch",
#)
# Rollouts
#config = config.rollouts(
#    num_rollout_workers=10,
#    num_envs_per_worker=1,
#)

In [10]:
register_env("finrl", env_creator)
config = PPOConfig()  
config = config.training(gamma=0.9, lr=0.01, kl_coeff=0.3) 
config = config.rollouts(num_rollout_workers=8)  
config = config.environment("finrl")  
config = config.framework(framework="torch")
algo = config.build()
#config = PPOConfig().training(lr=0.01).resources(num_gpus=0,num_gpus_per_worker=1).rollouts(num_rollout_workers=4).framework(framework="torch")
config['seed'] = 42

2023-04-02 18:08:30,284	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(RolloutWorker pid=1478) 2023-04-02 18:08:37,755	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=1478) 2023-04-02 18:08:37,755	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=1478) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=1478)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=1491) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Cas

In [11]:
register_env("finrl", env_creator)
config = ApexDQNConfig()
print(config.replay_buffer_config) 

config = config.training(replay_buffer_config=replay_config) 
config = config.rollouts(num_rollout_workers=8)  
config = config.environment("finrl")  
config = config.framework(framework="torch")
algo = config.build()

#config = DDPPOConfig().training(lr=0.01).resources(num_gpus=0,num_gpus_per_worker=1).rollouts(num_rollout_workers=4).framework(framework="torch")
#config['seed'] = 42

NameError: name 'ApexDQNConfig' is not defined

In [ ]:
# registering the environment to ray
#register_env("finrl", env_creator)
trainer = config.build(env="finrl") 
#trainer = DDPGConfig(env='finrl', config=config)

In [ ]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes = 25 
agent_name = 'ddppo'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    rwd = results[-1]['episode_reward_mean']
    print(f'Mean Rwd:{rwd}')
    if ep % 25 == 0:
        #cwd_checkpoint = "results/checkpoints/" + str(agent_name) + '_' + str(ep)
        cwd_checkpoint = "model/"  + str(agent_name)
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint saved in directory {cwd_checkpoint}")
bar.close()

In [ ]:
## Mean Rewards ### 
print(results[-1]['episode_reward_mean'])
#print(results[-1])

In [ ]:
# save the trained agent
#cwd_checkpoint = "model/" + str(agent_name) + '_' + str(ep)
cwd_checkpoint = "model/"
trainer.save(cwd_checkpoint)